In [1]:
!pip install selenium


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from datetime import datetime, timedelta
import pandas as pd
import calendar
import time

In [6]:
service = Service()
driver = webdriver.Chrome(service=service)
url_kita = "https://sitrack.id/live_track"
driver.get(url_kita)
time.sleep(3)

In [18]:
def scrape_monthly_weather(driver, month, year):
    # Bangun URL untuk bulan dan tahun yang spesifik
    base_url = "https://www.accuweather.com/en/id/jakarta/208971/"
    month_name = calendar.month_name[month].lower()
    url = f"{base_url}{month_name}-weather/208971?year={year}"

    driver.get(url)
    time.sleep(5)  # waktu untuk halaman untuk memuat sepenuhnya

    # Mendapatkan data tanggal, suhu tinggi, dan suhu rendah
    dates_elements = driver.find_elements(By.CSS_SELECTOR, "div.monthly-calendar > a")

    # Membuat list untuk menyimpan data
    data = []

    for date_element in dates_elements:
        date = date_element.find_element(By.CSS_SELECTOR, "div.monthly-panel-top > div").text
        high_temp = date_element.find_element(By.CSS_SELECTOR, "div.temp > div.high").text
        low_temp = date_element.find_element(By.CSS_SELECTOR, "div.temp > div.low").text
        # Menambahkan data ke list
        data.append({
            'Date': date,
            'High Temperature': high_temp,
            'Low Temperature': low_temp
        })

    return data
# Buat instance WebDriver untuk browser Chrome
driver = webdriver.Chrome()
# Membuat list untuk menyimpan semua data cuaca dari setiap bulan
all_weather_data = []
# Loop melalui setiap bulan dalam setahun terakhir
for month in range(1, 13):
    weather_data_month = scrape_monthly_weather(driver, month, 2023)
    # Tambahkan data cuaca bulanan ke dalam list all_weather_data
    all_weather_data.extend(weather_data_month)
# Membuat DataFrame dari semua data cuaca
df = pd.DataFrame(all_weather_data)
# Menyimpan DataFrame ke file Excel
excel_filename = "annual_weather_jakarta_2023.xlsx"
df.to_excel(excel_filename, index=False)
print(f"Data scraped and saved to {excel_filename}")
# Tutup browser setelah selesai scraping
driver.quit()

Data scraped and saved to annual_weather_jakarta_2023.xlsx


In [22]:
def scrape_monthly_weather(driver, month, year):
    # Bangun URL untuk bulan dan tahun yang spesifik
    base_url = "https://www.accuweather.com/en/id/jakarta/208971/"
    month_name = calendar.month_name[month].lower()
    url = f"{base_url}{month_name}-weather/208971?year={year}"

    driver.get(url)
    time.sleep(5)  # waktu untuk halaman untuk memuat sepenuhnya

    # Mendapatkan data tanggal, suhu tinggi, dan suhu rendah
    dates_elements = driver.find_elements(By.CSS_SELECTOR, "div.monthly-calendar > a")

    # Membuat list untuk menyimpan data
    data = []

    # Untuk melacak tanggal terakhir yang telah ditemukan
    last_date = None

    for date_element in dates_elements:
        date = date_element.find_element(By.CSS_SELECTOR, "div.monthly-panel-top > div").text
        high_temp = date_element.find_element(By.CSS_SELECTOR, "div.temp > div.high").text
        low_temp = date_element.find_element(By.CSS_SELECTOR, "div.temp > div.low").text

        # Konversi tanggal menjadi format tanggal yang sesuai
        date_obj = datetime.strptime(date, "%d")

        # Jika tanggal terakhir yang ditemukan kurang dari tanggal yang seharusnya
        if last_date is not None and date_obj != last_date + timedelta(days=1):
            # Tambahkan entri kosong untuk tanggal yang hilang
            for missing_date in range((last_date + timedelta(days=1)).day, date_obj.day):
                data.append({
                    'Date': f"{missing_date:02d}-{month:02d}-{year}",
                    'High Temperature': 'N/A',
                    'Low Temperature': 'N/A'
                })

        # Tambahkan data ke list
        data.append({
            'Date': f"{date_obj.day:02d}-{month:02d}-{year}",
            'High Temperature': high_temp,
            'Low Temperature': low_temp
        })

        # Perbarui tanggal terakhir
        last_date = date_obj

    return data

# Buat instance WebDriver untuk browser Chrome
driver = webdriver.Chrome()

# Membuat list untuk menyimpan semua data cuaca dari setiap bulan
all_weather_data = []

# Loop melalui setiap bulan dalam setahun terakhir
for month in range(1, 13):
    weather_data_month = scrape_monthly_weather(driver, month, 2023)
    # Tambahkan data cuaca bulanan ke dalam list all_weather_data
    all_weather_data.extend(weather_data_month)

# Membuat DataFrame dari semua data cuaca
df = pd.DataFrame(all_weather_data)

# Menyimpan DataFrame ke file Excel
excel_filename = "weather_jakarta_2023.xlsx"
df.to_excel(excel_filename, index=False)

print(f"Data scraped and saved to {excel_filename}")

# Tutup browser setelah selesai scraping
driver.quit()


Data scraped and saved to weather_jakarta_2023.xlsx
